In [2]:
import pandas as pd
import json
import os
# from pymol import cmd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# from statannotations.Annotator import Annotator
# from alphafetcher import AlphaFetcher

In [15]:
rm = pd.read_csv('./project_pipeline/data/md_rmsds.tsv', sep='\t')
pae = pd.read_csv('./project_pipeline/data/multi_domain_pae.tsv', sep='\t')
ob = pd.read_csv('./project_pipeline/data/obligate.tsv', sep='\t')
rmsd = pd.read_csv('./project_pipeline/data/classified_files.tsv', sep='\t')

ai_int = pd.read_csv('./project_pipeline/data/proteins_interface.tsv', sep='\t')
af_int = pd.read_csv('./project_pipeline/data/alphafold_interface.tsv', sep='\t')
md_int = pd.read_csv('./project_pipeline/data/md_proteins_interface.tsv', sep='\t')
mdaf_int = pd.read_csv('./project_pipeline/data/md_alphafold_interface.tsv', sep='\t')

af_hydro = pd.read_csv('./project_pipeline/data/ai_full_depth_structure_information.tsv', sep='\t')
mdaf_hydro = pd.read_csv('./project_pipeline/data/md_full_depth_structure_information.tsv', sep='\t')

ai_cl = pd.read_csv('./project_pipeline/data/ai_pdb_cluster_compared.tsv', sep='\t')
ai_cl = ai_cl[['uniprot', 'pdb', 'cluster', 'complex_rmsd', '2_comp']]

ai_uniprots = ai_int['uniprot'].tolist()
af_int = af_int[af_int['uniprot'].isin(ai_uniprots)]

ai_lens = ai_int[['uniprot', 'region_1_len', 'region_2_len']]
af_int = pd.merge(af_int, ai_lens, on='uniprot', how='left')

In [42]:
proteins = pd.read_csv('./project_pipeline/data/proteins.tsv', sep='\t')
cifs = './project_pipeline/data/input/Alphafold_cif/'
files = os.listdir(cifs)
cif_files = [f for f in files if '.cif' in f]
uniprot = 'P29476'
print([f for f in cif_files if uniprot in f])

for index, row in proteins.iterrows():
    uniprot = row['uniprot']
    fn = [f for f in cif_files if uniprot in f]
    proteins.at[index, 'af_filename'] = fn[0]

proteins.to_csv('./project_pipeline/data/proteins.tsv', sep='\t', index=False)

['F-P29476-F1-model_v3.cif']


Do we have any uniprots in md_rmsds that we don't have in multi_domain_pae?

In [16]:
best = rmsd.sort_values('complex_rmsd').groupby('uniprot').head(1).reset_index(drop=True)

best['matches'] = np.where(best['complex_rmsd'] <= 3, 'match', 'non_match')

best_cluster = ai_cl.sort_values('complex_rmsd').groupby('uniprot').head(1).reset_index(drop=True)

best_cluster['matches'] = np.where((best_cluster['complex_rmsd'] <= 3) & (best_cluster['2_comp'] <= 3), 'match', 'non_match')

In [17]:
pdb_states = rmsd[['pdb', 'state']]

In [21]:
cluster_matches = ai_cl[(ai_cl['complex_rmsd'] <= 3) & (ai_cl['2_comp'] <= 3)]
cluster_states = pd.merge(cluster_matches, pdb_states, on='pdb', how='left')
cluster_states

,uniprot,pdb,cluster,complex_rmsd,2_comp,state
0,P28482,4qp6,U10-009,2.522,2.925,Active
1,P28482,5v60,U100-001,1.729,2.722,Autoinhibited
2,P28482,5v60,U100-005,2.040,2.248,Autoinhibited
3,P28482,5v60,U100-004,1.997,1.929,Autoinhibited
4,P28482,5v60,000,1.397,2.953,Autoinhibited
...,...,...,...,...,...,...
1285,B5XAZ0,7bfl,U10-003,2.427,0.820,Autoinhibited
1286,B5XAZ0,7bfl,U100-002,1.987,0.877,Autoinhibited
1287,B5XAZ0,7bfl,003,2.418,0.876,Autoinhibited
1288,B5XAZ0,7bfl,007,2.281,1.522,Autoinhibited


In [22]:
#Do we have any proteins where the clusters match both states?
cluster_states.groupby('uniprot')['state'].nunique()

uniprot
B5XAZ0    1
P12931    1
P28482    2
P29350    1
P31016    1
P32527    1
P62826    1
Q02750    1
Q3KK31    0
Q8RT31    1
Q96BN8    2
Name: state, dtype: int64

In [23]:
Q96BN8 = rmsd[rmsd['uniprot'] == 'Q96BN8']
Q96BN8

,uniprot,pdb,region_1,region_2,complex_rmsd,percent_region_1,percent_region_2,2_aligned,2_comp,state,conformation,organism,date,notes,chain,af_filename
657,Q96BN8,3znv,330-340,129-341,1.666,100.000000,94.835681,1.662,1.208,Autoinhibited,Closed,HOMO SAPIENS,['2013-02-18'],NaN,A,F-Q96BN8-F1-model_v3.cif
658,Q96BN8,3znx,330-340,129-341,1.060,90.909091,91.079812,0.880,1.590,Active,Closed,HOMO SAPIENS,['2013-02-18'],NaN,A,F-Q96BN8-F1-model_v3.cif
659,Q96BN8,3znz,330-340,129-341,2.695,100.000000,90.610329,1.302,2.397,Active,Closed,HOMO SAPIENS,['2013-02-18'],NaN,A,F-Q96BN8-F1-model_v3.cif
660,Q96BN8,4ksj,330-340,129-341,1.224,100.000000,88.262911,0.882,1.104,Autoinhibited,Closed,Homo sapiens,['2013-05-17'],NaN,A,F-Q96BN8-F1-model_v3.cif
661,Q96BN8,4ksk,330-340,129-341,1.216,100.000000,90.140845,0.932,1.165,NaN,Closed,Homo sapiens,['2013-05-17'],NaN,A,F-Q96BN8-F1-model_v3.cif
662,Q96BN8,4ksl,330-340,129-341,2.087,100.000000,91.079812,1.657,2.432,NaN,Closed,Homo sapiens,['2013-05-17'],NaN,A,F-Q96BN8-F1-model_v3.cif
663,Q96BN8,5oe7,330-340,129-341,2.113,100.000000,91.549296,1.281,2.383,Active,Closed,Homo sapiens,['2017-07-07'],NaN,A,F-Q96BN8-F1-model_v3.cif
664,Q96BN8,6i9c,330-340,129-341,1.236,100.000000,90.610329,1.132,1.472,Active,Closed,Homo sapiens,['2018-11-22'],NaN,A,F-Q96BN8-F1-model_v3.cif
665,Q96BN8,6sak,330-340,129-341,1.240,100.000000,94.366197,0.934,1.802,NaN,Closed,Homo sapiens,['2019-07-17'],NaN,A,F-Q96BN8-F1-model_v3.cif


In [402]:
best = best[['uniprot', 'pdb', 'matches', 'complex_rmsd', '2_comp', 'state']]
best_cluster = best_cluster[['uniprot', 'pdb', 'matches', 'complex_rmsd', '2_comp', 'cluster']]

compare = pd.merge(best, best_cluster, on='uniprot', how='inner', suffixes=('_best', '_cluster'))

In [403]:
compare

,uniprot,pdb_best,matches_best,complex_rmsd_best,2_comp_best,state,pdb_cluster,matches_cluster,complex_rmsd_cluster,2_comp_cluster,cluster
0,Q3KK31,3pjx,match,1.031,1.130,NaN,3pjw,match,1.691,2.303,U100-008
1,Q96BN8,3znx,match,1.060,1.590,Active,3znx,match,1.147,1.604,U100-004
2,O60741,5u6p,match,1.182,1.290,Active,5u6o,non_match,19.891,24.167,U10-000
3,Q02750,3zly,match,1.211,1.034,Autoinhibited,6q0j,match,1.087,0.702,000
4,Q9Y6K1,4u7t,match,1.245,1.268,Active,4u7p,non_match,8.059,8.484,U100-006
5,P12931,4k11,match,1.254,2.074,Autoinhibited,7yqe,match,1.402,2.585,U10-007
6,P22681,4a49,match,1.351,1.498,Active,4a49,non_match,5.325,8.563,U100-003
7,P53042,4ja7,match,1.380,12.407,Autoinhibited,4ja9,non_match,2.132,11.179,U100-000
8,Q8RT31,4c7p,match,1.401,1.762,Autoinhibited,4c7p,match,1.732,2.575,U10-005
9,P29350,3ps5,match,1.444,1.765,Active,3ps5,match,2.199,2.209,U100-007


In [425]:
match = best[(best['complex_rmsd'] <= 3) & (best['2_comp'] <= 3)]

match['state'].value_counts()

Autoinhibited    25
Active           13
Name: state, dtype: int64

In [423]:
match_uniprots = match['uniprot'].unique().tolist()

In [424]:
non_match = best[~best['uniprot'].isin(match_uniprots)]
non_match['state'].value_counts()

Autoinhibited    39
Active           12
Name: state, dtype: int64

In [427]:
md_cl = pd.read_csv('./project_pipeline/data/md_pdb_cluster_compared.tsv', sep='\t')

obli_uniprots = ob['uniprot'].tolist()

md_cl = md_cl[md_cl['uniprot'].isin(obli_uniprots)]
md_cl['uniprot'].nunique()

7

In [407]:
cl = pd.read_csv('./project_pipeline/data/ai_pdb_cluster_compared.tsv', sep='\t')

only_u = cl[cl['cluster'].str.contains('U')]

min_only_u = only_u.sort_values('complex_rmsd').groupby('uniprot').head(1).reset_index(drop=True)
matched_min_only_u = min_only_u[min_only_u['complex_rmsd'] <= 3]
matched_min_only_u.shape

(14, 24)

In [408]:
non_u = cl[~cl['cluster'].str.contains('U')]

min_non_u = non_u.sort_values('complex_rmsd').groupby('uniprot').head(1).reset_index(drop=True)
matched_min_non_u = min_non_u[min_non_u['complex_rmsd'] <= 3]
matched_min_non_u.shape

(8, 24)

In [409]:
td = pd.read_csv('./project_pipeline/data/md_pdb_cluster_compared.tsv', sep='\t')

only_u = td[td['cluster'].str.contains('U')]

min_only_u = only_u.sort_values('complex_rmsd').groupby('uniprot').head(1).reset_index(drop=True)
matched_min_only_u = min_only_u[min_only_u['complex_rmsd'] <= 3]
matched_min_only_u.shape

(25, 24)

In [410]:
non_u = td[~td['cluster'].str.contains('U')]

min_non_u = non_u.sort_values('complex_rmsd').groupby('uniprot').head(1).reset_index(drop=True)
matched_min_non_u = min_non_u[min_non_u['complex_rmsd'] <= 3]
matched_min_non_u.shape

(10, 24)

In [5]:
single = pd.read_csv('./project_pipeline/data/single_domain_pae.tsv', sep='\t')

single = single[single['region'].str.contains(',')]
single.isna().sum()

uniprot      0
region       0
mean_pae    34
dtype: int64